In [1]:
%run interpreter.ipynb

..........................................................................
----------------------------------------------------------------------
Ran 74 tests in 0.488s

OK
................................................................................
----------------------------------------------------------------------
Ran 80 tests in 0.529s

OK
..........................................................................................
----------------------------------------------------------------------
Ran 90 tests in 0.710s

OK
........................................................................................................
----------------------------------------------------------------------
Ran 104 tests in 1.430s

OK


In [23]:
def identify(statement):
    node = interpreter.parse(statement)
    if isinstance(node, Tree):
        if node.data == 'implication':
            condition, conclusion = node.children  
            return (condition, conclusion)
        else:
            raise Exception("Couldn't find condition and conclusion! Statement is not implication!")
    else:
        raise Exception("Statement is not Tree!")

In [27]:
def diagnose(node, expected_value):
    if interpreter.interpret(node) != expected_value:
        return node, expected_value

In [28]:
def explain(statement):
    condition, conclusion = identify(statement)
    if interpreter.interpret(condition) == True and interpreter.interpret(conclusion) == False:
        print("Good counterexample!")
        return True
    else:
        if interpreter.interpret(condition) == False:
            return diagnose(condition, True)
        else:
            return diagnose(conclusion, False)

In [29]:
relation1 = HomogeneousRelation({(1,2),(2,3)},{1,2,3}, True)
relation2 = HomogeneousRelation({(1,1),(2,2)},{1,2,3}, True)

interpreter = Interpreter({'R':relation1,'S':relation2}, {1,2,3}, True)
statement = "R is transitive -> S is reflexive"

print(explain(statement))

(Tree('is', [Token('RELATION', 'R'), Tree('transitive', [])]), True)


In [19]:
condition, conclusion = identify(statement)

In [20]:
print(f"condition: {condition}, {interpreter.interpret(condition)}")
print(f"conclusion: {conclusion}, {interpreter.interpret(conclusion)}")

condition: Tree('is', [Token('RELATION', 'R'), Tree('transitive', [])]), True
conclusion: Tree('is', [Token('RELATION', 'S'), Tree('reflexive', [])]), True


In [260]:
def explain(statement, interpreter):
    condition, conclusion = identify(statement)
    if interpreter.interpret(condition) == False:
        print(f"You made mistake in condition! Condition can not be False!")
        return (condition, True)
    elif interpreter.interpret(conclusion) == True:
        print(f"You made mistake in conclusion! Conclusion can not be True!")
        return (conclusion, False)
    else:
        print(f"There is nothing wrong! You found good counterexample!")
        return None

In [261]:
explain(statement, interpreter)

You made mistake in condition! Condition can not be False!


(Tree('conjunction', [Tree('is', [Token('RELATION', 'R'), Tree('transitive', [])]), Tree('is', [Token('RELATION', 'S'), Tree('transitive', [])])]),
 True)

In [262]:
def explain_more(node, desired_boolean_value, interpreter):
    if isinstance(node, Tree):
        if node.data == 'negation':
            return explain_more(node.children[0], not desired_boolean_value, interpreter)
        elif len(node.children) == 2:
            left, right = node.children 
            if node.data == 'equal':
                if (interpreter.interpret(left) == interpreter.interpret(right)) != desired_boolean_value:
                    return [explain_more(left, desired_boolean_value, interpreter), \
                            explain_more(right, desired_boolean_value, interpreter)]
            elif node.data == 'not_equal':
                if (interpreter.interpret(left) != interpreter.interpret(right)) != desired_boolean_value:
                    return [explain_more(left, desired_boolean_value, interpreter), \
                            explain_more(right, desired_boolean_value, interpreter)]
            elif node.data == 'greater_than':
                if (interpreter.interpret(left) > interpreter.interpret(right)) != desired_boolean_value:
                    return [explain_more(left, desired_boolean_value, interpreter), \
                            explain_more(right, desired_boolean_value, interpreter)]
            elif node.data == 'lower_than':
                if (interpreter.interpret(left) < interpreter.interpret(right)) != desired_boolean_value:
                    return [explain_more(left, desired_boolean_value, interpreter), \
                            explain_more(right, desired_boolean_value, interpreter)]
            elif node.data == 'greater_equal':
                if (interpreter.interpret(left) >= interpreter.interpret(right)) != desired_boolean_value:
                    return [explain_more(left, desired_boolean_value, interpreter), \
                            explain_more(right, desired_boolean_value, interpreter)]
            elif node.data == 'lower_equal':
                if (interpreter.interpret(left) <= interpreter.interpret(right)) != desired_boolean_value:
                    return [explain_more(left, desired_boolean_value, interpreter), \
                            explain_more(right, desired_boolean_value, interpreter)]
            elif node.data == 'conjunction':
                if (interpreter.interpret(left) and interpreter.interpret(right)) != desired_boolean_value:
                    return [explain_more(left, desired_boolean_value, interpreter), \
                            explain_more(right, desired_boolean_value, interpreter)]
            elif node.data == 'disjunction':
                if (interpreter.interpret(left) or interpreter.interpret(right)) != desired_boolean_value:
                    return [explain_more(left, desired_boolean_value, interpreter), \
                            explain_more(right, desired_boolean_value, interpreter)]
                    
            elif node.data == 'implication':
                if ((not interpreter.interpret(left)) or interpreter.interpret(right)) != desired_boolean_value:
                    return [explain_more(left, desired_boolean_value, interpreter), \
                            explain_more(right, desired_boolean_value, interpreter)]
            elif node.data == 'intersection':
                if (interpreter.interpret(left) & interpreter.interpret(right)) != desired_boolean_value:
                    return [explain_more(left, desired_boolean_value, interpreter), \
                            explain_more(right, desired_boolean_value, interpreter)]
            elif node.data == 'union':
                if (interpreter.interpret(left) | interpreter.interpret(right)) != desired_boolean_value:
                    return [explain_more(left, desired_boolean_value, interpreter), \
                            explain_more(right, desired_boolean_value, interpreter)]
            elif node.data == 'difference':
                if (interpreter.interpret(left) - interpreter.interpret(right)) != desired_boolean_value:
                    return [explain_more(left, desired_boolean_value, interpreter), \
                            explain_more(right, desired_boolean_value, interpreter)]
            elif node.data == 'symmetric_difference':
                if (interpreter.interpret(left) ^ interpreter.interpret(right)) != desired_boolean_value:
                    return [explain_more(left, desired_boolean_value, interpreter), \
                            explain_more(right, desired_boolean_value, interpreter)]
            elif node.data == 'multiplication':
                if (interpreter.interpret(left) * interpreter.interpret(right)) != desired_boolean_value:
                    return [explain_more(left, desired_boolean_value, interpreter), \
                            explain_more(right, desired_boolean_value, interpreter)]
            elif node.data == 'composition':
                if (interpreter.interpret(left).composition(interpreter.interpret(right))) != desired_boolean_value:
                    return [explain_more(left, desired_boolean_value, interpreter), \
                            explain_more(right, desired_boolean_value, interpreter)]
            elif node.data == 'is' or node.data == 'is_not':
                if node.data == 'is_not':
                    desired_boolean_value = not desired_boolean_value
                if right.data == 'lattice':            
                    if interpreter.interpret(left).is_lattice() != desired_boolean_value:
                        #print(f"Statement\n{node.pretty()} should be {desired_boolean_value}, but is {not desired_boolean_value}!")
                        return (node, desired_boolean_value)
                    else:
                        return None             
                elif right.data == 'total_order':
                    if interpreter.interpret(left).is_total_order() != desired_boolean_value:
                        #print(f"Statement\n{node.pretty()} should be {desired_boolean_value}, but is {not desired_boolean_value}!")
                        return (node, desired_boolean_value)
                    else:
                        return None
                elif right.data == 'reflexive':
                    if interpreter.interpret(left).is_reflexive() != desired_boolean_value:
                        #print(f"Statement\n{node.pretty()} should be {desired_boolean_value}, but is {not desired_boolean_value}!")
                        return (node, desired_boolean_value)
                    else:
                        return None
                elif right.data == 'symmetric':
                    if interpreter.interpret(left).is_symmetric() != desired_boolean_value:
                        #print(f"Statement\n{node.pretty()} should be {desired_boolean_value}, but is {not desired_boolean_value}!")
                        return (node, desired_boolean_value)
                    else:
                        return None
                elif right.data == 'asymmetric':
                    if interpreter.interpret(left).is_assymmetric() != desired_boolean_value:
                        #print(f"Statement\n{node.pretty()} should be {desired_boolean_value}, but is {not desired_boolean_value}!")
                        return (node, desired_boolean_value)
                    else:
                        return None
                elif right.data == 'antisymmetric':
                    if interpreter.interpret(left).is_antisymmetric() != desired_boolean_value:
                        #print(f"Statement\n{node.pretty()} should be {desired_boolean_value}, but is {not desired_boolean_value}!")
                        return (node, desired_boolean_value)
                    else:
                        return None
                elif right.data == 'transitive':
                    if interpreter.interpret(left).is_transitive() != desired_boolean_value:
                        #print(f"Statement\n{node.pretty()} should be {desired_boolean_value}, but is {not desired_boolean_value}!")
                        return (node, desired_boolean_value)
                    else:
                        return None
        else:
            raise Exception("ERROR! Unexpected input!")
    else:
        raise Exception("Can't explain more! You have bad input! I always expect Tree!")
    

In [263]:
node, desired_boolean_value = explain(statement, interpreter)

You made mistake in condition! Condition can not be False!


In [264]:
return_value = explain_more(node, desired_boolean_value, interpreter)

In [265]:
for p in return_value:
    print(p)

(Tree('is', [Token('RELATION', 'R'), Tree('transitive', [])]), True)
None


In [266]:
len(return_value)

2

In [267]:
return_value

[(Tree('is', [Token('RELATION', 'R'), Tree('transitive', [])]), True), None]

In [269]:
return_value = explain_more(node, desired_boolean_value, interpreter)
while len(return_value) == 2:
    a, b = return_value
    print()
    if b == True or b == False or b is None:
        print(return_value)
        break
    else:
        print(a)
        return_value = b


[(Tree('is', [Token('RELATION', 'R'), Tree('transitive', [])]), True), None]
